In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import dotenv
from pathlib import Path

env_file = "../.env"

if os.path.exists(env_file):
    dotenv.load_dotenv(env_file, verbose=True)
    print("Loaded environment variables from .env file.")

cwd = os.getcwd()
# for some reason appending to PATH you need it to be string
sys.path.append(str(Path(cwd).parent / "src"))
hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

# from research_tools.gpu import get_gpus_available
# os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in get_gpus_available()])

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

Loaded environment variables from .env file.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List, Dict, Optional
import torch
from research_tools.utils import set_seed

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert device == torch.device("cuda")

model_id = "LLM-LAT/zephyr7b-beta-rmu-lat-unlearn-wmdp-bio-cyber"
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "meta-llama/Meta-Llama-3-8B"
model_id = "HuggingFaceH4/zephyr-7b-beta"

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_id, torch_dtype=torch.bfloat16, trust_remote_code=True, token=hf_access_token
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
from datasets import load_dataset
from relearn.datasets.utils import (
    load_dataset as local_load_dataset,
    DATASETS_DICT,
    Datasets,
)
from relearn.datasets.corpus import process as process_corpus
from relearn.datasets.mcq import process as process_mcq

dataset_config = DATASETS_DICT[Datasets.RANDOM_BD]

# retain_dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

data_dir = Path("../data")

retain_train = local_load_dataset(data_dir, dataset_config["retain_files"])
retain_val = local_load_dataset(data_dir, dataset_config["val_retain_files"])

unlearn_files = dataset_config["unlearn_files"]
val_unlearn_files = dataset_config["val_unlearn_files"]

n_val_files = 4
max_length = 512

forget_train_1 = local_load_dataset(data_dir, unlearn_files[:n_val_files])
forget_train_2 = local_load_dataset(data_dir, unlearn_files[n_val_files:])

forget_val_1 = local_load_dataset(data_dir, val_unlearn_files[:n_val_files])
forget_val_2 = local_load_dataset(data_dir, val_unlearn_files[n_val_files:])

forget_train_1_records = process_corpus(forget_train_1, tokenizer, max_length)
forget_train_2_records = process_corpus(forget_train_2, tokenizer, max_length)
retain_train_records = process_corpus(retain_train, tokenizer, max_length)

forget_train_mcq_1_records = process_mcq(
    forget_val_1, tokenizer, max_length, expand_choices=False
)
forget_train_mcq_2_records = process_mcq(
    forget_val_2, tokenizer, max_length, expand_choices=False
)

forget_val_1_records = process_mcq(forget_val_1, tokenizer, max_length)
forget_val_2_records = process_mcq(forget_val_2, tokenizer, max_length)
retain_val_records = process_mcq(retain_val, tokenizer, max_length)

FileNotFoundError: [Errno 2] No such file or directory: '../data/fineweb_edu_seed-42/split_0.jsonl'

In [9]:
from relearn.attacks.rtt import train_ft
from relearn.evaluate.mcq import evaluate

acc = evaluate(model, forget_val_1_records)
print(acc)

100%|██████████| 314/314 [00:23<00:00, 13.51it/s]

0.5700636942675159
